In [13]:
import pandas as pd
import numpy as np
from os import path

In [ ]:
data_path = path.abspath("../data/US_Superstore_data.xls")
data = pd.read_excel(data_path)
# Country -> only have United States values
data = data.drop(columns=['Row ID', 'Order ID', 'Customer Name', 'Postal Code', 'Product ID', 'Product Name', 'Customer ID', 'Order Date', 'Ship Date', 'Country'])
data

,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...
9989,Second Class,Consumer,Miami,Florida,South,Furniture,Furnishings,25.2480,3,0.20,4.1028
9990,Standard Class,Consumer,Costa Mesa,California,West,Furniture,Furnishings,91.9600,2,0.00,15.6332
9991,Standard Class,Consumer,Costa Mesa,California,West,Technology,Phones,258.5760,2,0.20,19.3932
9992,Standard Class,Consumer,Costa Mesa,California,West,Office Supplies,Paper,29.6000,4,0.00,13.3200


Normalização

In [18]:
data['Sales']=(data['Sales']-data['Sales'].mean())/data['Sales'].std()
data['Profit']=(data['Profit']-data['Profit'].mean())/data['Profit'].std()
data

,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,0.051508,2,0.00,0.056590
1,Second Class,Consumer,Henderson,Kentucky,South,Furniture,Chairs,0.805593,3,0.00,0.815013
2,Second Class,Corporate,Los Angeles,California,West,Office Supplies,Labels,-0.345350,2,0.00,-0.092997
3,Standard Class,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,1.167630,5,0.45,-1.757397
4,Standard Class,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,-0.332919,2,0.20,-0.111587
...,...,...,...,...,...,...,...,...,...,...,...
9989,Second Class,Consumer,Miami,Florida,South,Furniture,Furnishings,-0.328298,3,0.20,-0.104816
9990,Standard Class,Consumer,Costa Mesa,California,West,Furniture,Furnishings,-0.221258,2,0.00,-0.055595
9991,Standard Class,Consumer,Costa Mesa,California,West,Technology,Phones,0.046078,2,0.20,-0.039544
9992,Standard Class,Consumer,Costa Mesa,California,West,Office Supplies,Paper,-0.321315,4,0.00,-0.065470
